In [205]:
import pandas as pd
import numpy as np
import yfinance as yf
from dgl import function as fn

In [3]:
etfs = pd.read_csv('etf_with_stock.csv')

In [5]:
etf_target = pd.read_csv('etf 수익률 및 1년간 코스피 관련 지표.csv')

In [9]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [34]:
tickers = etfs[['etf코드','날짜']].drop_duplicates()

In [35]:
etf_target

,etf코드,날짜,수익률,1년간 코스피 수익률,1년간 코스피 std
0,KR7069500007,20240329,0.285791,0.108916,0.009729
1,KR7069660009,20240329,-0.182700,0.108916,0.009729
2,KR7091160002,20240329,1.821579,0.108916,0.009729
3,KR7091170001,20240329,0.333033,0.108916,0.009729
4,KR7091180000,20240329,0.010257,0.108916,0.009729
...,...,...,...,...,...
1269,KR7333980001,20200331,-0.139820,-0.180331,0.015094
1270,KR7337120000,20200331,-0.168515,-0.180331,0.015094
1271,KR7337140008,20200331,-0.137616,-0.180331,0.015094
1272,KR7337150007,20200331,-0.225855,-0.180331,0.015094


In [77]:
stocks = etfs[['종목코드','날짜']].drop_duplicates()
date_dic = {
    20240329 : ["2023-03-31" , "2024-03-29"],
    20230331 : ["2022-03-31" , "2023-03-31"],
    20220331 : ["2021-03-31" , "2022-03-31"],
    20210331 : ["2020-03-31" , "2021-03-31"],
    20200331 : ["2019-03-29" , "2020-03-31"]
}

stocks['종가들'] = None
for index, row in stocks.iterrows():
    code = row['종목코드'] + '.KS'
    date = row['날짜']
    start = date_dic[date][0]
    end = date_dic[date][1]
    
    closes = yf.download(code,
                    start = start,
                    end = end)
    stocks.at[index,'종가들'] = closes['Close'].values

    

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [104]:
stocks['종가들'].apply(len).value_counts()

종가들
0      2314
246    1478
244     826
248     819
249     719
253      82
247      49
251      49
252      41
87        3
64        3
99        2
148       2
112       2
192       2
245       1
150       1
114       1
147       1
130       1
142       1
72        1
86        1
76        1
113       1
183       1
125       1
94        1
171       1
213       1
93        1
157       1
100       1
104       1
139       1
156       1
211       1
85        1
202       1
205       1
163       1
22        1
119       1
238       1
233       1
129       1
232       1
151       1
127       1
121       1
67        1
164       1
207       1
40        1
154       1
89        1
81        1
218       1
90        1
Name: count, dtype: int64

In [109]:
stocks['로그 수익률'] = None
min_len = 244

for index, row in stocks.iterrows():
    close_price = row['종가들']
    if np.isnan(close_price).any():
        continue
    if len(close_price) < min_len:
        continue
    
    log_returns = np.log(np.array(close_price[1:min_len]) / np.array(close_price[:min_len-1]))
    
    stocks.at[index, '로그 수익률'] = log_returns.tolist()

In [110]:
stocks

,종목코드,날짜,종가들,로그 수익률
0,005930,20240329,"[64000.0, 63100.0, 63600.0, 63900.0, 62300.0, ...","[-0.01416231381250436, 0.007892700798908983, 0..."
1,000660,20240329,"[88600.0, 87200.0, 84500.0, 84600.0, 83800.0, ...","[-0.015927526696101306, -0.03145279655180573, ..."
2,005380,20240329,"[184500.0, 183600.0, 183800.0, 186700.0, 18510...","[-0.004889985294191792, 0.0010887317351966558,..."
3,068270,20240329,"[149800.0, 150000.0, 159000.0, 158600.0, 15950...","[0.0013342230131366622, 0.058268908123975824, ..."
4,005490,20240329,"[368000.0, 392000.0, 367000.0, 373500.0, 36250...","[0.06317890162153167, -0.06589999173589231, 0...."
...,...,...,...,...
120762,002690,20200331,"[2864.891357421875, 2851.39892578125, 2878.383...","[-0.0047207038232059085, 0.009419227267825977,..."
120763,002420,20200331,"[6479.205078125, 6393.9521484375, 6498.1499023...","[-0.013245262923440256, 0.016164939191093523, ..."
120764,015260,20200331,"[3244.024658203125, 3231.177001953125, 3237.60...","[-0.003968269648978906, 0.001986103218698862, ..."
121396,071950,20200331,"[1370.0, 1370.0, 1375.0, 1785.0, 1805.0, 1770....","[0.0, 0.0036429912785010087, 0.260964684113067..."


# 그래프 구축하기

In [ ]:
code = 'KR7069500007'
date = 20240329

graph = etfs[(etfs['etf코드'] == code) & (etfs['날짜'] == date)]

n = len(graph)
graph_new = dgl.DGLGraph(num_nodes=n)
graph_new.add_nodes(n)

node_features = []

for i in range(n):
    # f2 = graph.iloc[i]['시장구분'] #str
    # f3 = graph.iloc[i]['업종명'] #str
    # f4 = graph.iloc[i]['주식 종류'] #str
    node_feat_list = [graph.iloc[i]['구성비중'], graph.iloc[i]['PER'], graph.iloc[i]['PBR'], 
                      graph.iloc[i]['PCR'], graph.iloc[i]['PSR'], graph.iloc[i]['로그 수익률']]
    node_features.append(node_feat_list)

node_features_tensor = torch.tensor(node_features, dtype=torch.float32)

graph_new.ndata['feat'] = node_features_tensor


for i in range(n):
    
    stock_i = graph.iloc[i]['종목코드']
    year_log_return_i = graph.iloc[i]['로그 수익률']
    log_return_i = pd.Series(stocks[(stocks['종목코드'] == stock_i) & (stocks['날짜'] == date)]['로그 수익률'].values[0])
    
    for j in range(i + 1, n):
        stock_j = graph.iloc[j]['종목코드']
        year_log_return_j = graph.iloc[j]['로그 수익률']
        log_return_j = pd.Series(stocks[(stocks['종목코드'] == stock_j) & (stocks['날짜'] == date)]['로그 수익률'].values[0])

        value=0
        if log_return_j is None or log_return_i is None:
            max_return = abs(max(year_log_return_i,year_log_return_j))   
            min_return  = abs(min(year_log_return_i,year_log_return_j))
            value = min_return / max_return
        else:
            value = log_return_j.corr(log_return_i)

        
        if abs(value) > 0.3:
            graph_new.add_edges(i, j)  
            graph_new.add_edges(j, i)
            graph_new.edges[i, j].data['weight'] = torch.tensor([value])
            graph_new.edges[j, i].data['weight'] = torch.tensor([value])

print(graph_new)

In [162]:
graphs = []
labels = []

for index, row in tickers.iterrows():
    code = row['etf코드']
    date = row['날짜']
    graph = etfs[(etfs['etf코드'] == code) & (etfs['날짜'] == date)]

    n = len(graph)
    graph_new = dgl.DGLGraph(num_nodes=n)
    graph_new.add_nodes(n)

    node_features = []

    for i in range(n):
        # f2 = graph.iloc[i]['시장구분'] #str
        # f3 = graph.iloc[i]['업종명'] #str
        # f4 = graph.iloc[i]['주식 종류'] #str
        node_feat_list = [graph.iloc[i]['구성비중'], graph.iloc[i]['PER'], graph.iloc[i]['PBR'], 
                          graph.iloc[i]['PCR'], graph.iloc[i]['PSR'], graph.iloc[i]['로그 수익률']]
        node_features.append(node_feat_list)
    
    node_features_tensor = torch.tensor(node_features, dtype=torch.float32)
    
    graph_new.ndata['feat'] = node_features_tensor
    
    
    for i in range(n):
        
        stock_i = graph.iloc[i]['종목코드']
        year_log_return_i = graph.iloc[i]['로그 수익률']
        log_return_i = pd.Series(stocks[(stocks['종목코드'] == stock_i) & (stocks['날짜'] == date)]['로그 수익률'].values[0])
        
        for j in range(i + 1, n):
            stock_j = graph.iloc[j]['종목코드']
            year_log_return_j = graph.iloc[j]['로그 수익률']
            log_return_j = pd.Series(stocks[(stocks['종목코드'] == stock_j) & (stocks['날짜'] == date)]['로그 수익률'].values[0])
    
            value=0
            if log_return_j is None or log_return_i is None:
                max_return = abs(max(year_log_return_i,year_log_return_j))   
                min_return  = abs(min(year_log_return_i,year_log_return_j))
                value = min_return / max_return
            else:
                value = log_return_j.corr(log_return_i)
    
            
            if abs(value) > 0.3:
                graph_new.add_edges(i, j)  
                graph_new.add_edges(j, i)
                graph_new.edges[i, j].data['weight'] = torch.tensor([value])
                graph_new.edges[j, i].data['weight'] = torch.tensor([value])

    label = etf_target[(etf_target['etf코드'] == code) & (etf_target['날짜'] == date)]['수익률'].values[0]

    print(label)
    print(graph_new)

    graphs.append(graph_new)
    labels.append(label)
    

0.2857913060967358
Graph(num_nodes=201, num_edges=4970,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
-0.1827000347133622
Graph(num_nodes=189, num_edges=4462,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
1.8215789966307896
Graph(num_nodes=51, num_edges=24,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
0.3330326065236555
Graph(num_nodes=11, num_edges=68,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
0.0102567410445852
Graph(num_nodes=21, num_edges=144,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
0.5785949664561698
Gra

C:\Users\user\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\user\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


0.4786707189026062
Graph(num_nodes=671, num_edges=74270,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
-0.4942234288221372
Graph(num_nodes=98, num_edges=1944,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
-0.4725306608396262
Graph(num_nodes=198, num_edges=6750,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
-0.3696985991023137
Graph(num_nodes=199, num_edges=9538,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
0.0409729728983818
Graph(num_nodes=110, num_edges=2364,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
0.3054983073

In [194]:
count = 0
remove_indexs = []
for i in range(len(graphs)):
    try:
        if graphs[i].edata['weight'] is not None:
            count += 1
    except Exception as e:
        remove_indexs.append(i)
print(remove_indexs)
print(count)
print(len(graphs))

[72, 89, 162, 187, 190, 257, 258, 280, 286, 287, 293, 294, 296, 297, 304, 310, 311, 316, 323, 326, 328, 330, 333, 486, 496, 521, 524, 591, 592, 614, 620, 621, 784, 809, 812, 968, 1040, 1065, 1068, 1250]
1234
1274


In [195]:
graphs_final = []
for i in range(len(graphs)):
    if i not in remove_indexs:
        graphs_final.append(graphs[i])

[Graph(num_nodes=201, num_edges=4970,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)}), Graph(num_nodes=189, num_edges=4462,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)}), Graph(num_nodes=51, num_edges=24,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)}), Graph(num_nodes=11, num_edges=68,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)}), Graph(num_nodes=21, num_edges=144,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)}), Graph(num_nodes=11, num_edges=68,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      ed

In [201]:
label_final = []
for i in range(len(labels)):
    if i not in remove_indexs:
        label_final.append(labels[i])

In [226]:
class GCNWithNodeAndEdgeFeatures(nn.Module):
    def __init__(self, in_node_feats, in_edge_feats, hidden_size):
        super(GCNWithNodeAndEdgeFeatures, self).__init__()
        self.conv1 = GraphConv(in_node_feats, hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(hidden_size, 1, allow_zero_in_degree=True)  # 출력 노드가 하나이므로 1로 설정
        self.edge_embed = nn.Linear(in_edge_feats, hidden_size)

    def forward(self, g, node_features, edge_features):
        node_feats = F.relu(self.conv1(g, node_features))
        edge_feats = F.relu(self.edge_embed(edge_features.float()))
        g.ndata['h'] = node_feats
        g.edata['h'] = edge_feats
        g.update_all(fn.u_mul_e('h', 'h', 'm'), fn.sum('m', 'h_Neigh'))
        node_feats_updated = g.ndata.pop('h_Neigh')
        x = self.conv2(g, node_feats_updated)
        
        g.ndata['x'] = x
        graph_pred = dgl.mean_nodes(g, 'x')
        return graph_pred.squeeze()
        # return x.squeeze(1)  # 1차원 노드를 평탄화하여 반환

edges = []
nodes = []
for graph in graphs_final:
    edge_weights = graph.edata['weight']
    edges.append(edge_weights)
    node_features = graph.ndata['feat']
    nodes.append(node_features)


C:\Users\user\AppData\Local\Temp\ipykernel_3132\1630312543.py:44: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(logits, label_tensor)


In [229]:
import random
combined_data = list(zip(graphs_final, nodes, label_final, edges))
random.shuffle(combined_data)

# 데이터를 분할합니다.
train_size = int(0.8 * len(combined_data))
train_data = combined_data[:train_size]
test_data = combined_data[train_size:]

In [230]:
# 훈련 데이터를 추출합니다.
train_graphs_final, train_nodes, train_label_final, train_edges = zip(*train_data)

# 테스트 데이터를 추출합니다.
test_graphs_final, test_nodes, test_label_final, test_edges = zip(*test_data)

In [231]:
# 모델을 초기화합니다.
model = GCNWithNodeAndEdgeFeatures(in_node_feats=6, in_edge_feats=1, hidden_size=6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 모델을 훈련합니다.
model.train()
for epoch in range(100):
    for graph, node_features, label, edge_features in zip(train_graphs_final, train_nodes, train_label_final, train_edges):
        logits = model(graph, node_features, edge_features.unsqueeze(1))
        logits = logits.view(-1, 1) 
        if isinstance(label, (int, float)):
            label_tensor = torch.tensor([label], dtype=torch.float32)
        else:
            label_tensor = torch.tensor(label, dtype=torch.float32).unsqueeze(1)
        loss = F.mse_loss(logits, label_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

C:\Users\user\AppData\Local\Temp\ipykernel_3132\1978374031.py:15: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(logits, label_tensor)


In [234]:
# 모델을 평가합니다.
model.eval()
test_losses = []
for graph, node_features, label, edge_features in zip(test_graphs_final, test_nodes, test_label_final, test_edges):
    with torch.no_grad():
        logits = model(graph, node_features, edge_features.unsqueeze(1))
        logits = logits.view(-1, 1) 
        if isinstance(label, (int, float)):
            label_tensor = torch.tensor([[label]], dtype=torch.float32)
        else:
            label_tensor = torch.tensor(label, dtype=torch.float32).unsqueeze(1)
        loss = F.mse_loss(logits, label_tensor)
        test_losses.append(loss.item())

In [240]:
# 테스트 손실의 평균을 계산합니다.
sum=0
for loss in test_losses:
    sum+=loss

avg_test_loss = sum / len(test_losses)
print("Average test loss:", avg_test_loss)

Average test loss: 13.19259513064328


In [241]:
test_losses

[0.5355934500694275,
 1.171066403388977,
 0.0054609146900475025,
 0.024051476269960403,
 0.21583911776542664,
 0.44146567583084106,
 0.18873652815818787,
 0.7310378551483154,
 0.37377673387527466,
 0.17707231640815735,
 0.0801258236169815,
 5.821857484988868e-05,
 0.25527423620224,
 0.15996293723583221,
 0.06853536516427994,
 0.09588851779699326,
 0.32699647545814514,
 0.5948854088783264,
 2.7454407245386392e-05,
 0.0018500854494050145,
 0.18869511783123016,
 0.2049778401851654,
 0.7503492832183838,
 0.1869126409292221,
 0.2677771747112274,
 0.05679523944854736,
 0.28908422589302063,
 0.1056804433465004,
 0.43934211134910583,
 0.6866361498832703,
 0.39270034432411194,
 5.905129909515381,
 0.04184141382575035,
 0.33155709505081177,
 0.36123010516166687,
 0.1272137612104416,
 0.1709570288658142,
 0.16993515193462372,
 2981.8134765625,
 0.01156823243945837,
 0.41284629702568054,
 0.5276244282722473,
 0.885679304599762,
 0.0656074658036232,
 1.1001290082931519,
 0.44967925548553467,
 0.328

# 테스트 코드

In [120]:
one_hot = pd.get_dummies(etfs[['시장구분','업종명','주식 종류']])
one_hot = one_hot.map(lambda x : 1 if x == True else 0)
one_hot.columns

Index(['시장구분_KOSDAQ', '시장구분_KOSPI', '시장구분_현금', '업종명_IT부품', '업종명_건설', '업종명_건설업',
       '업종명_광업', '업종명_금속', '업종명_금융', '업종명_기계', '업종명_기계·장비', '업종명_기타금융',
       '업종명_기타서비스', '업종명_기타제조', '업종명_기타제조업', '업종명_농업, 임업 및 어업', '업종명_디지털컨텐츠',
       '업종명_반도체', '업종명_방송서비스', '업종명_보험', '업종명_비금속', '업종명_비금속광물', '업종명_서비스업',
       '업종명_섬유·의류', '업종명_섬유의복', '업종명_소프트웨어', '업종명_숙박·음식', '업종명_오락·문화',
       '업종명_운송', '업종명_운송장비·부품', '업종명_운수장비', '업종명_운수창고업', '업종명_유통', '업종명_유통업',
       '업종명_은행', '업종명_음식료·담배', '업종명_음식료품', '업종명_의료·정밀기기', '업종명_의료정밀',
       '업종명_의약품', '업종명_인터넷', '업종명_일반전기전자', '업종명_전기가스업', '업종명_전기전자', '업종명_정보기기',
       '업종명_제약', '업종명_종이·목재', '업종명_종이목재', '업종명_증권', '업종명_철강금속', '업종명_출판·매체복제',
       '업종명_컴퓨터서비스', '업종명_통신서비스', '업종명_통신업', '업종명_통신장비', '업종명_현금', '업종명_화학',
       '주식 종류_보통주', '주식 종류_우선주', '주식 종류_현금'],
      dtype='object')

In [121]:
one_hot.columns.size

60

In [123]:
code = 'KR7069500007'
date = 20240329
graph = etfs[(etfs['etf코드'] == code) & (etfs['날짜'] == date)]

src_nodes = []
dst_nodes = []
edge_weight = []
features = []

n = len(graph)

for i in range(n):

    f1 = graph.iloc[i]['구성비중']
    # f2 = graph.iloc[i]['시장구분'] #str
    # f3 = graph.iloc[i]['업종명'] #str
    # f4 = graph.iloc[i]['주식 종류'] #str
    f5 = graph.iloc[i]['PER']
    f6 = graph.iloc[i]['PBR']
    f7 = graph.iloc[i]['PCR']
    f8 = graph.iloc[i]['PSR']
    f9 = graph.iloc[i]['로그 수익률']
    
    feature = [f1,f5,f6,f7,f8,f9]
    features.append(feature)
    
    stock_i = graph.iloc[i]['종목코드']
    year_log_return_i = graph.iloc[i]['로그 수익률']
    log_return_i = pd.Series(stocks[(stocks['종목코드'] == stock_i) & (stocks['날짜'] == date)]['로그 수익률'].values[0])
    
    for j in range(i+1,n):

        stock_j = graph.iloc[j]['종목코드']
        year_log_return_j = graph.iloc[j]['로그 수익률']
        log_return_j = pd.Series(stocks[(stocks['종목코드'] == stock_j) & (stocks['날짜'] == date)]['로그 수익률'].values[0])

        value=0
        if log_return_j is None or log_return_i is None:
            max_return = abs(max(year_log_return_i,year_log_return_j))   
            min_return  = abs(min(year_log_return_i,year_log_return_j))
            value = min_return / max_return
        else:
            value = log_return_j.corr(log_return_i)
        
        if value > 0.3 or value < -0.3:
            src_nodes.append(i)
            dst_nodes.append(j)
            src_nodes.append(j)
            dst_nodes.append(i)
            
            edge_weight.append(value)
            edge_weight.append(value)

src = torch.tensor(src_nodes)
dst = torch.tensor(dst_nodes)
graph_new = dgl.graph((src_nodes, dst_nodes))

node_features = torch.tensor(features)
graph_new.ndata['feat'] = node_features

edge_weights = torch.tensor(edge_weight)
graph_new.edata['weight'] = edge_weights

label = etf_target[(etf_target['etf코드'] == code) & (etf_target['날짜'] == date)]['수익률'].values[0]

print(label)
print(graph_new)


DGLError: Expect number of features to match number of nodes (len(u)). Got 201 and 200 instead.

In [156]:
code = 'KR7069500007'
date = 20240329

graph = etfs[(etfs['etf코드'] == code) & (etfs['날짜'] == date)]

n = len(graph)
graph_new = dgl.DGLGraph(num_nodes=n)
graph_new.add_nodes(n)

node_features = []

for i in range(n):
    # f2 = graph.iloc[i]['시장구분'] #str
    # f3 = graph.iloc[i]['업종명'] #str
    # f4 = graph.iloc[i]['주식 종류'] #str
    node_feat_list = [graph.iloc[i]['구성비중'], graph.iloc[i]['PER'], graph.iloc[i]['PBR'], 
                      graph.iloc[i]['PCR'], graph.iloc[i]['PSR'], graph.iloc[i]['로그 수익률']]
    node_features.append(node_feat_list)

node_features_tensor = torch.tensor(node_features, dtype=torch.float32)

graph_new.ndata['feat'] = node_features_tensor


for i in range(n):
    
    stock_i = graph.iloc[i]['종목코드']
    year_log_return_i = graph.iloc[i]['로그 수익률']
    log_return_i = pd.Series(stocks[(stocks['종목코드'] == stock_i) & (stocks['날짜'] == date)]['로그 수익률'].values[0])
    
    for j in range(i + 1, n):
        stock_j = graph.iloc[j]['종목코드']
        year_log_return_j = graph.iloc[j]['로그 수익률']
        log_return_j = pd.Series(stocks[(stocks['종목코드'] == stock_j) & (stocks['날짜'] == date)]['로그 수익률'].values[0])

        value=0
        if log_return_j is None or log_return_i is None:
            max_return = abs(max(year_log_return_i,year_log_return_j))   
            min_return  = abs(min(year_log_return_i,year_log_return_j))
            value = min_return / max_return
        else:
            value = log_return_j.corr(log_return_i)

        
        if abs(value) > 0.3:
            graph_new.add_edges(i, j)  
            graph_new.add_edges(j, i)
            graph_new.edges[i, j].data['weight'] = torch.tensor([value])
            graph_new.edges[j, i].data['weight'] = torch.tensor([value])

print(graph_new)

Graph(num_nodes=201, num_edges=4970,
      ndata_schemes={'feat': Scheme(shape=(6,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})
